In [1]:
import json
import re

In [2]:
def parse_log_line(line):
    match = re.search(r'/(\S+): negotiation completed. Configuration: (\{.*?\}).*?Rounds: (\d+).*?Result: (\S+)', line)
    if match:
        robot = match.group(1)
        config = json.loads(match.group(2))
        rounds = int(match.group(3))
        result = match.group(4)
        return ((robot, config["goal"]), rounds, result)
    return None

def load_log_file(file_path):
    log_entries = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            parsed = parse_log_line(line)
            if parsed:
                key, rounds, result = parsed
                log_entries[key] = (rounds, result, line.strip())
    return log_entries

def compare_logs(file1, file2):
    log1 = load_log_file(file1)
    log2 = load_log_file(file2)
    
    for key in log1.keys() & log2.keys():
        rounds1, result1, line1 = log1[key]
        rounds2, result2, line2 = log2[key]
        
        if rounds1-rounds2 < -1 or rounds1-rounds2 > 1 or result1 != result2:
            print(f"Discrepanza trovata per la configurazione: {key}")
            print(f"File 1: {line1}")
            print(f"File 2: {line2}\n")
    
    missing_in_file2 = log1.keys() - log2.keys()
    missing_in_file1 = log2.keys() - log1.keys()
    
    if missing_in_file2:
        print("Configurations in File 1 alone:")
        for key in missing_in_file2:
            print(log1[key][2])
    
    if missing_in_file1:
        print("Configurations in File 2 alone:")
        for key in missing_in_file1:
            print(log2[key][2])

In [3]:

file1 = "../results/simulation-onrobot.log"
file2 = "../results/simulation-offrobot.log"
compare_logs(file1, file2)